# Lab 03: Fun with `pandas`!

Below are some exercises to get you working with `pandas` to manipulate data. As always, get as far as you can, and ask for help when you need it! Your teacher (me), you instructor, and your classmates are all here to help each other get better at coding. Getting the code to work is important, but do also take the time to make sure you understand what the commands are doing. This time, (with the exception of the Stroop challenge), all I've given you is the code to download the data. Then you are on your own. For the Stroop challenge, I gave the you code for the first step—after that, it's up to you :-)

## Music sales challenge

Write a script that:

1. Combines the tables of best-selling physical singles and best-selling digital singles on the Wikipedia page "List_of_best-selling_singles"
2. Adds a column which marks whether each row is from the list of physical singles or digital singles
3. Outputs the artist and single name for the year you were born. If there is no entry for that year, take the closest year after you were born.
4. Outputs the artist and single name for the year you were 15 years old.

In [4]:
# Starter code...

%pip install lxml
import pandas as pd

rawdata = pd.read_html("https://en.wikipedia.org/wiki/List_of_best-selling_singles") # this gives a list of data frames
# print(rawdata)


# Inspect the data - find which table corresponds to digital and physical singles
# rawdata is a list of DataFrames, so we can inspect each DataFrame
for i, df in enumerate(rawdata):
    print(f"Table {i}:")
    #print(df.head(), '\n')  # Print the first few rows of each table to identify them

physical_singles = rawdata[0]
digital_singles = rawdata[3]

# add row to identify type of single
physical_singles['Type'] = 'Physical'
digital_singles['Type'] = 'Digital'

singles_merged = pd.concat([digital_singles,physical_singles], ignore_index=False)
#print(singles_merged)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 21.8 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.
Table 0:
Table 1:
Table 2:
Table 3:
Table 4:
Table 5:
Table 6:
Table 7:
Table 8:
Table 9:
Table 10:
Table 11:
Table 12:
Table 13:


In [5]:
# Clean the sales column so only numbers
print(singles_merged.columns) # check column names
singles_merged['Sales'] = singles_merged['Sales (in millions)'] # add new column called Sales
singles_merged['Sales'] = singles_merged['Sales'].replace(r'[^\d.]', '', regex=True).astype(float) # clean 

# sort by descending
singles_merged = singles_merged.sort_values(by="Sales", ascending = False)

# re-index list
singles_merged.reset_index(drop=True, inplace=True)

print(singles_merged)

Index(['Artist', 'Single', 'Released', 'Sales (in millions)', 'Source',
       'Type'],
      dtype='object')
                                  Artist  \
0                            Bing Crosby   
1                             Ed Sheeran   
2      Luis Fonsi featuring Daddy Yankee   
3                             Elton John   
4                Rihanna featuring Drake   
5                            Bing Crosby   
6                             Tino Rossi   
7                Bill Haley & His Comets   
8                        Whitney Houston   
9          The Chainsmokers and Coldplay   
10                            Ed Sheeran   
11    Wiz Khalifa featuring Charlie Puth   
12     The Chainsmokers featuring Halsey   
13                                 Adele   
14                        USA for Africa   
15                         Elvis Presley   
16      Mark Ronson featuring Bruno Mars   
17                         Billie Eilish   
18                            Ed Sheeran   
19        

In [6]:
# output artist and single from year I was born - 2000. 
target_year = 2000

singles_2000 = singles_merged[singles_merged['Released'] == target_year]

if not singles_2000.empty:
    print(f"Singles from the year {target_year}:")
    print(singles_2000[['Artist', 'Single']])
else:
    # Step 3: If no singles from 2000, search for the closest year with singles
    print(f"No singles found from the year {target_year}. Searching for the closest year...")
# Initialize variables for the closest year search
    found_singles = False
    offset = 1  # Start checking 1 year before/after the target year
    
    while not found_singles:
        # Check the previous year and next year alternately
        year_before = target_year - offset
        year_after = target_year + offset
        
        # Check if there are singles from year_before or year_after
        singles_before = singles_merged[singles_merged['Released'] == year_before]
        singles_after = singles_merged[singles_merged['Released'] == year_after]
        
        if not singles_before.empty:
            print(f"Singles from the year {year_before}:")
            print(singles_before[['Artist', 'Single']])
            found_singles = True  # Exit the loop when found
        elif not singles_after.empty:
            print(f"Singles from the year {year_after}:")
            print(singles_after[['Artist', 'Single']])
            found_singles = True  # Exit the loop when found
        
        # Increment offset to check further years if no singles are found
        offset += 1

No singles found from the year 2000. Searching for the closest year...
Singles from the year 1997:
         Artist                                             Single
3    Elton John  "Something About the Way You Look Tonight"/"Ca...
22  Celine Dion                              "My Heart Will Go On"


In [7]:
# find singles and artists from the year when you were 15

age_15 = 2015 # year I was 15

singles_2015 = singles_merged[singles_merged['Released'] == age_15] # locate release year
print(singles_2015[['Artist', 'Single', 'Released']]) # see single and artist


                                Artist               Single  Released
11  Wiz Khalifa featuring Charlie Puth      "See You Again"      2015
16    Mark Ronson featuring Bruno Mars        "Uptown Funk"      2015
18                          Ed Sheeran  "Thinking Out Loud"      2015


## Space challenge

1. Make a single dataframe that combines the space missions from the 1950's to the 2020's
2. Write a script that returns the year with the most launches
3. Write a script that returns the most common month for launches
4. Write a script that ranks the months from most launches to fewest launches


In [8]:
# Starter code...

#%pip install lxml
import pandas as pd

spacedata = pd.read_html("https://en.wikipedia.org/wiki/Timeline_of_Solar_System_exploration")
# print(spacedata) # outputs list of various data frames

# combine all tables into data frame
len(spacedata) # check how many tables there are
space_all = pd.concat(spacedata, ignore_index = True)


In [18]:
# Script to return year with the most launches

# Step 1: Convert the 'Launch Date' column to datetime
space_all['Launch date'] = pd.to_datetime(space_all['Launch date'], format='%d %B %Y')
# Step 2: Isolate the year from the datetime column and create new column in df
space_all['Year'] = space_all['Launch date'].dt.year

missionsperyear = space_all.groupby('Year').size()
# Find the year with the most missions
year_with_most_missions = missionsperyear.idxmax()
most_missions_count = missionsperyear.max()

print(f"The year with the most released singles is {year_with_most_missions}, with {most_missions_count} singles released.")
    

The year with the most released singles is 1965.0, with 12 singles released.


In [28]:
# Write a script that returns the most common month for launches

import calendar

# Isolate the month from the datetime column and create new column in df
space_all['Month'] = space_all['Launch date'].dt.month

missionspermonth = space_all.groupby('Month').size()
# Find the months with the most missions
month_with_most_missions = missionspermonth.idxmax()
month_most_missions_count = missionspermonth.max()

# Ensure month_with_most_missions is an integer
month_with_most_missions = int(month_with_most_missions)

# Convert the numerical month to its textual representation
month_name_with_most_missions = calendar.month_name[month_with_most_missions]

# Print the result in a sentence
print(f"The month with the most released singles is {month_name_with_most_missions}, with {month_most_missions_count} singles released.")

The month with the most released singles is November, with 30 singles released.


In [92]:
# Write a script that ranks the months from most launches to fewest launches

#print(missionspermonth)
#print(type(missionspermonth))

# Convert missionspermonth (Series) to a DataFrame with two columns: 'Month' and 'Count'
missionspermonth_df = missionspermonth.reset_index(name='Count')

# Ensure the Month column is an integer (if it has decimals)
missionspermonth_df['Month'] = missionspermonth_df['Month'].fillna(0).astype(int)

# Convert the 'Count' column to integer to remove any decimal points
missionspermonth_df['Count'] = missionspermonth_df['Count'].astype(int)

# Convert numerical months to month names (only once)
missionspermonth_df['Month'] = missionspermonth_df['Month'].apply(lambda x: calendar.month_name[x])

# Rename the columns to something more descriptive
missionspermonth_df = missionspermonth_df.rename(columns={'Month': 'Month', 'Count': 'Number of Missions'})

# Check the DataFrame to ensure the columns are renamed
print(missionspermonth_df)


# Sort data frame according to descending number of launches
df_sorted = missionspermonth_df.sort_values(by=['Number of Missions'], ascending = False)

df_sorted.reset_index()


        Month  Number of Missions
0     January                  19
1    February                  14
2       March                  15
3       April                  13
4         May                  17
5        June                  14
6        July                  21
7      August                  27
8   September                  22
9     October                  22
10   November                  30
11   December                  19


,index,Month,Number of Missions
0,10,November,30
1,7,August,27
2,9,October,22
3,8,September,22
4,6,July,21
5,11,December,19
6,0,January,19
7,4,May,17
8,2,March,15
9,1,February,14


## Supervillain challenge

1. Write a script that combines the tables showing supervillain debuts from the 30's through the 2010's
2. Write a script that ranks each decade in terms of how many supervillains debuted in that decade
3. Write a script that ranks the different comics companies in terms of how many supervillains they have, and display the results in a nice table (pandas dataframe)

In [ ]:
supervillain = pd.read_html("https://en.wikipedia.org/wiki/List_of_comic_book_supervillain_debuts")

In [103]:
# Combine tables of all supervillains 

#print(supervillain) # outputs list of various data frames

# combine all tables into data frame
len(supervillain) # check how many tables there are
supervillain_all = pd.concat(supervillain, ignore_index = True)
print(supervillain_all.columns)

Index([                 0,                  1, 'Character / Team',
           'Year Debuted',          'Company',        'Creator/s',
       'First Appearance'],
      dtype='object')


In [119]:
# Rank each decade based on how many supervillains deubted in that decade

# function to determine decade
def get_decade(year):
    return (year // 10) * 10 if year > 0 else None

# Set Year Debuted to integer
supervillain_all['Year Debuted'] = (
    supervillain_all['Year Debuted']
    .replace(r'[^\d]', '', regex=True)  # Replace non-digit characters
    .fillna(0.1)                          # Fill NaN values with 0.1
    .astype(int)                        # Convert to integer
)

# Apply the function to create a new column for decades
supervillain_all['Decade'] = supervillain_all['Year Debuted'].apply(get_decade)

# Count the number of debuts in each decade
decade_counts = supervillain_all.groupby('Decade').size().reset_index(name = "Count")

# Sort the results by decade
decade_counts = decade_counts.sort_values(by='Count', ascending = False)
decade_counts


OverflowError: Python int too large to convert to C long

In [117]:
# Write a script that ranks the different comics companies in terms of how many 
# supervillains they have, and display the results in a pandas df

# Count the number of supervillains in each company
company_villains = supervillain_all.groupby('Company').size().reset_index(name='Count')

company_villains = pd.DataFrame(company_villains)

company_villains = company_villains.sort_values(by='Count', ascending = False)
company_villains.reset_index(inplace = True)
company_villains

,index,Company,Count
0,1,DC,338
1,9,Marvel,264
2,5,Fawcett Comics/DC,6
3,2,Dark Horse,5
4,6,Image,5
5,3,Disney/Hyperion,4
6,10,Marvel/Timely,4
7,4,Eternity,3
8,0,Comico,1
9,7,Image Comics,1


## Stroop challenge

Every year between 2015 and 2021, the students in my Language, Cognition, and the Brain course participated in a version of the Stroop task. Using a stopwatch (ok, using their phones), they recorded how fast they could say a list of things (either reading or naming colors or color words). The column names mean "Reading with No Interference", "Naming with Interference", "Naming with No Interference", and "Reading with Interference". The times are in seconds.

### Stroop challenge 1: 
Transform these data from wide format to long format, so that the result is a dataframe with
- 1 column named "Participant_id" with a unique number for each participant (you can use the row indices)
- 1 column named "Year" with the year data
- 1 column named "Task" that shows which task they were doing
- 1 column named "RT" that shows their response time

In [149]:
# Starter code 1...

df = pd.read_csv("https://raw.githubusercontent.com/ethanweed/Stroop/master/Stroop-raw-over-the-years.csv")
df.head()

# add a ID column
df['Participant_id'] = range(1, len(df) + 1) 

df.head()

# Step 1: Melt the DataFrame
df_long = df.melt(
    id_vars=['Year', 'Participant_id'],
    var_name='Task',  # Name for the new variable column
    value_name='RT'        # Name for the response time column
)

# Step 3: Sort the DataFrame
df_long.sort_values(by=['Participant_id', 'Year'], inplace=True)

# Reset index 
df_long.reset_index(drop=True, inplace=True)

df_long.head()

,Year,Participant_id,Task,RT
0,2015,1,Reading_NoInt,4.16
1,2015,1,Naming_Int,6.76
2,2015,1,Naming_NoInt,4.45
3,2015,1,Reading_Int,4.65
4,2015,2,Reading_NoInt,4.35


## Stroop challenge 2 (Advanced!!!):

Make a new dataframe which shows the mean response time (in seconds) for each task for each year.